In [98]:
import os
import re
import csv
import statistics
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [6]:
# test files for one participant and one gaze file
eye_files = os.listdir('../data/168/gaze')
eye_file = pd.read_csv('../data/168/gaze/168_gaze_reading_createServerChooser.csv')

In [89]:
ref = pd.read_csv('./word_coordinates_final/createServerChooser_boxes.csv') # reading in bounding boxes
boxes = pd.DataFrame()

for i in range(len(ref)): # creating shapes for each word in the file
    x = ref.loc[i, 'tobii_x']
    y = ref.loc[i, 'tobii_y']
    w = ref.loc[i, 'tobii_width']
    h = ref.loc[i, 'tobii_height']
    word = '{word}.{num}'.format(word=ref.loc[i, 'word'], num=ref.loc[i, 'occurrence'])
    # Creating a shape for each word based on its coordinates
    new_tangle = pd.Series({word: Polygon([(x, y), (x+w, y), (x, y+h), (x+w, y+h)])}) 
    boxes = pd.concat([boxes, new_tangle]) # adding this to a new data structure 

In [19]:
# resetting column names
eye_file.columns = ['participant_id', 'function_name', 'function_id', 'system_timestamp', 'device_timestamp', 'valid_gaze_left', 'valid_gaze_right', 'gaze_left_eye', 'gaze_right_eye', 'valid_pd_left', 'valid_pd_right', 'gaze_left', 'gaze_right']

In [111]:
test = gpd.GeoSeries(boxes[0]) # temp name for data structure of boxes

public.0                     POLYGON ((0.01146 0.12500, 0.04740 0.12500, 0....
void.0                       POLYGON ((0.05000 0.12500, 0.07500 0.12500, 0....
createServerChooser.0        POLYGON ((0.07865 0.12500, 0.19844 0.12500, 0....
MainMenuBar.0                POLYGON ((0.03333 0.15000, 0.09792 0.15000, 0....
mainMenuBar.0                POLYGON ((0.10104 0.15000, 0.16510 0.15000, 0....
=.0                          POLYGON ((0.16823 0.15000, 0.17656 0.15000, 0....
MainMenuBar.getInstance.0    POLYGON ((0.17969 0.15000, 0.32708 0.15000, 0....
int.0                        POLYGON ((0.03333 0.20000, 0.05260 0.20000, 0....
nodeId.0                     POLYGON ((0.05625 0.20000, 0.09219 0.20000, 0....
=.1                          POLYGON ((0.09531 0.20000, 0.10365 0.20000, 0....
dbMgr.getNodeId.0            POLYGON ((0.10677 0.20000, 0.20365 0.20000, 0....
String.0                     POLYGON ((0.20781 0.20000, 0.24375 0.20000, 0....
cbNE.getSelectedItem.0       POLYGON ((0.25833 0.200

In [153]:
newfile = '168_localizing_gaze_test_createServerChooser.csv'
new_eye_df = pd.DataFrame()

for i in range(len(eye_file)): # iterating through eyetracking file and localizing gaze to a shape
    gaze_left = eye_file.loc[i, 'gaze_left_eye']
    gaze_right = eye_file.loc[i, 'gaze_right_eye']
    temp_left = re.split(r'[(,\)]', gaze_left) # tuple is a string, so using regex to get the numbers
    temp_right = re.split(r'[(,\)]', gaze_right)
    # averaging right and left gaze points for one gaze point
    gaze_point = (statistics.fmean([float(temp_left[1]), float(temp_right[1])]), 
                 statistics.fmean([float(temp_left[2]), float(temp_right[2])]))
    if pd.isna(gaze_point[0]) or pd.isna(gaze_point[1]): # if either eye is invalid
        continue
    else:
        gaze_point = [Point(gaze_point[0], gaze_point[1])]
    pnts = gpd.GeoDataFrame(geometry=gaze_point)
    
    # confusing, but basically this below line returns false/true whether the point is in a shape
    pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in test.items()}) 
    temp = pd.DataFrame(pnts)
    #temp.to_csv('168_localizing_gaze_test_createServerChooser.csv')
    old_row = eye_file.loc[i, :]
    new_row = pd.concat([old_row, temp.T]) # adding this true/false information to the original row
    new_eye_df = pd.concat([new_eye_df, new_row], axis=1)
new_eye_df = new_eye_df.T
new_eye_df.to_csv(newfile)

In [ ]:

#From ChatGPT
new_eye_df = pd.DataFrame()

for i, row in eye_file.iterrows():
    gaze_left = row['gaze_left_eye']
    gaze_right = row['gaze_right_eye']
    temp_left = [float(x) for x in re.findall(r'\d+\.\d+', gaze_left)]
    temp_right = [float(x) for x in re.findall(r'\d+\.\d+', gaze_right)]

    gaze_point = ((temp_left[0] + temp_right[0]) / 2,
                  (temp_left[1] + temp_right[1]) / 2)
    
    if pd.isnull(gaze_point[0]) or pd.isnull(gaze_point[1]):
        continue
    else:
        gaze_point = [Point(gaze_point[0], gaze_point[1])]
    
    pnts = gpd.GeoDataFrame(geometry=gaze_point)
    pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in test.items()})
    temp = pd.DataFrame(pnts)
    new_row = row.join(temp.T)
    new_eye_df = new_eye_df.append(new_row)

new_eye_df = new_eye_df.reset_index(drop=True)
new_eye_df.to_csv(newfile, index=False)